In [10]:
#from __future__ import division, print_function, absolute_import
import os
import shutil
import tensorflow as tf
import pandas as pd
import tensorflow.contrib.learn as learn
import numpy as np
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import multiprocessing


def get_model_dir(name,erase):
    base_path = os.path.join(".","dnn")
    model_dir = os.path.join(base_path,name)
    os.makedirs(model_dir,exist_ok=True)
    return model_dir

tf.logging.set_verbosity(tf.logging.INFO)
    
#save init (?)
#sess = tf.Session()
#sess.run(tf.global_variables_initializer())
#builder = tf.saved_model.builder.SavedModelBuilder("./model")
#sess.run(tf.global_variables_initializer())

print(tf.VERSION)

df = pd.read_csv('../NNNormalizeData-out.csv')

np.random.seed(42) # always shuffle the same way 
df = df.reindex(np.random.permutation(df.index)) # shuffle examples 
df.reset_index(inplace=True, drop=True)

inputs = []
target = []

y=0;    
for x in df.columns:
    if y != 35 :
        inputs.append(x)
    else :
        target.append(x)
    y+=1

total_inputs,total_output = df.as_matrix(inputs).astype(np.float32),df.as_matrix([target]).astype(np.int32)

train_inputs, test_inputs, train_output, test_output = train_test_split(total_inputs, total_output, test_size=0.2, random_state=42)

print("Total number of examples %d" %(total_inputs.shape[0]+1))
print("Train number of examples %d" %(train_inputs.shape[0]+1))
print("Test number of examples %d" %(test_inputs.shape[0]+1))


feature_columns = [tf.contrib.layers.real_valued_column("", dimension=train_inputs.shape[1])]
#target_column = [tf.contrib.layers.real_valued_column("output", dimension=train_output.shape[1])]
model_dir = get_model_dir('ModelSave',True)

training_steps = 20000
save_checkpoints_steps=400 # used to run the early stopping monitor 
early_stopping_rounds=2000   # after 50 SPETS if "loss" doesn't improve stop the train
batch_size=(train_inputs.shape[0]+1)/10 # train number of examples / 10 - 10 batch = 1 epoch

# The hyperparameters specified here will be searched.  Every combination.


classifier = learn.DNNClassifier(hidden_units=[100, 50, 20], n_classes=5
                                 ,model_dir= model_dir
                                 #,optimizer=tf.train.ProximalAdagradOptimizer(
                                 #     learning_rate=0.01)
                                 #     l1_regularization_strength=0.001
                                 #     )
                                 ,config=tf.contrib.learn.RunConfig(save_checkpoints_steps=save_checkpoints_steps
                                                                    ,save_checkpoints_secs=None)
                                 ,feature_columns=feature_columns)


#early stopping using validation monitor 
print("configuring early stopping")
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    x=test_inputs,
    y=test_output,
    #every_n_steps=200,  # when to run the monitor - not working - forcing with save_checkpoints_steps
    early_stopping_metric="loss",         #"accuracy" of "loss"
    early_stopping_metric_minimize=True,     #False Maximize accuracy (True is minimize applied to loss)  
    early_stopping_rounds=early_stopping_rounds)

print("Fit will start...")
# Startup grid search
threads = multiprocessing.cpu_count()
print("Using {} cores.".format(threads))

param_grid = {
    'learning_rate': [0.1, 0.01, 0.001]
    ,'batch_size': [64, 128, 256, batch_size]
    #,'steps': [training_steps]
}

grid_classifier = GridSearchCV(classifier,verbose=True, n_jobs=threads,scoring='accuracy', 
                             param_grid=param_grid ,fit_params={'monitor':validation_monitor})



#classifier = learn.SKCompat(classifier) # For Sklearn compatibility
grid_classifier.fit(train_inputs, train_output)
print("Fit is finish...")


#print (classifier.get_variable_names()) 

#Save Model into saved_model.pbtxt file (possible to Load in Java)
#tfrecord_serving_input_fn = tf.contrib.learn.build_parsing_serving_input_fn(layers.create_feature_spec_for_parsing(feature_columns))  
#classifier.export_savedmodel(export_dir_base="test", serving_input_fn = tfrecord_serving_input_fn,as_text=True)


# Measure accuracy
pred = list(classifier.predict(test_inputs, as_iterable=True))
#pred = list(classifier.predict(test_inputs))
score = metrics.accuracy_score(test_output, pred)
print("Final score: {}".format(score))

accuracy_score = classifier.evaluate(x=test_inputs,
                                     y=test_output)["accuracy"]

print('Accuracy: {0:f}'.format(accuracy_score))

# test individual samples 
sample_1 = np.array( [[0.37671986791414125,0.28395908337619136,-0.0966095873607713,-1.0,0.06891621389763203,-0.09716678086712205,0.726029084013637,4.984689881073479E-4,-0.30296253267499107,-0.16192917054985334,0.04820256230479658,0.4951319883569152,0.5269983894210499,-0.2560313828048315,-0.3710980821053321,-0.4845867212612598,-0.8647234314469595,-0.6491591208322198,-1.0,-0.5004549422844073,-0.9880910165770813,0.5540293108747256,0.5625990251930839,0.7420121698556554,0.5445551415657979,0.4644276850235627,0.7316976292340245,0.636690006814346,0.16486621649984112,-0.0466018967678159,0.5261100063227044,0.6256168612312738,-0.544295484930702,0.379125782517193,0.6959368575211544]], dtype=float)
sample_2 = np.array( [[1.0,0.7982741870963959,1.0,-0.46270838239235024,0.040320274521029376,0.443451913224413,-1.0,1.0,1.0,-1.0,0.36689718911339564,-0.13577379160035796,-0.5162916256414466,-0.03373651520104648,1.0,1.0,1.0,1.0,0.786999801054777,-0.43856035121103853,-0.8199093927945158,1.0,-1.0,-1.0,-0.1134921695894473,-1.0,0.6420892436196663,0.7871737734493178,1.0,0.6501788845358409,1.0,1.0,1.0,-0.17586627413625022,0.8817194210401085]], dtype=float)

pred = list(classifier.predict(sample_2, as_iterable=True))
print("Prediction for sample_2 is:{} ".format(pred))

pred = list(classifier.predict_proba(sample_2, as_iterable=True))
print("Prediction for sample_2 is:{} ".format(pred))



C:\Users\birinhos\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\birinhos\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


1.0.1
Total number of examples 24300
Train number of examples 19440
Test number of examples 4861
INFO:tensorflow:Using config: {'_task_type': None, '_save_checkpoints_steps': 400, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_id': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000BDCCAC8>, '_save_summary_steps': 100, '_environment': 'local', '_master': '', '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': None}
configuring early stopping
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Fit will start...
Using 4 cores.
Fitting 3 folds for each of 12 candidates, totalling 36 fits


TypeError: __init__() got an unexpected keyword argument 'params'